In [ ]:
!pip install torch torchvision transformers pillow scikit-learn pandas tqdm matplotlib

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageFilter
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from transformers import CLIPProcessor, CLIPModel
from tqdm.notebook import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import kagglehub

# Re-download the dataset
dataset_path = kagglehub.dataset_download('jangedoo/utkface-new')

print(f"Dataset downloaded to: {dataset_path}")

In [ ]:
import os

# Path where the dataset is downloaded
dataset_path = "/kaggle/input/utkface-new/UTKFace"

# List the files in the directory
print(os.listdir(dataset_path))

In [ ]:
def categorize_age(age):
    if age <= 10:
        return "0-10"
    elif age <= 20:
        return "11-20"
    elif age <= 30:
        return "21-30"
    elif age <= 40:
        return "31-40"
    elif age <= 50:
        return "41-50"
    else:
        return "50+"

In [ ]:

# Extract file paths and labels
image_paths, age_labels, gender_labels = [], [], []

for filename in tqdm(os.listdir(dataset_path)):
    image_path = os.path.join(dataset_path, filename)
    temp = filename.split('_')
    age = categorize_age(int(temp[0]))
    gender = int(temp[1])  # 0: Male, 1: Female
    image_paths.append(image_path)
    age_labels.append(age)
    gender_labels.append(gender)

# Create DataFrame
df = pd.DataFrame({'image': image_paths, 'age': age_labels, 'gender': gender_labels})
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle the DataFrame

# Gender dictionary
gender_dic = {0: 'Male', 1: 'Female'}

In [ ]:
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import os

def detect_face_yunet(image_path, target_size=(384, 384)):
    """Detects a face using YuNet and preprocesses it safely."""

    # Ensure the file exists
    if not os.path.exists(image_path):
        print(f"⚠️ File not found: {image_path}")
        return None

    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️ Could not load image: {image_path}")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Ensure RGB format

    # Initialize YuNet
    yunet = cv2.FaceDetectorYN.create(
        model='/content/drive/MyDrive/Age Prediction/Models/face_detection_yunet_2023mar.onnx',
        config='',
        input_size=(320, 320),
        score_threshold=0.6,  # Lower threshold to detect more faces
        nms_threshold=0.3,
        top_k=5000
    )

    yunet.setInputSize((img.shape[1], img.shape[0]))

    # Detect faces
    _, faces = yunet.detect(img)

    if faces is None or len(faces) == 0:
        print(f"⚠️ No face detected in: {image_path}")
        return None

    # Ensure YuNet returns at least (x, y, w, h)
    face = faces[0][:4] if faces.shape[1] >= 4 else None

    if face is None:
        print(f"⚠️ Face detection failed for: {image_path}")
        return None

    x, y, w, h = map(int, face)  # Convert to int for cropping

    # ✅ **Clamp negative values to 0** ✅
    x = max(0, x)
    y = max(0, y)

    # Ensure valid crop
    if w <= 0 or h <= 0 or x >= img.shape[1] or y >= img.shape[0]:
        print(f"⚠️ Invalid face coordinates: {x}, {y}, {w}, {h}")
        return None

    cropped_face = img[y:y + h, x:x + w]

    # Resize safely
    if cropped_face.size == 0:
        print(f"⚠️ Empty cropped face for: {image_path}")
        return None

    cropped_face = cv2.resize(cropped_face, target_size)

    return cropped_face


def preprocess_image(image_path, target_size=(384, 384)):
    """
    Preprocess the image with resizing, enhancement, noise reduction, etc.
    """
    img = Image.open(image_path)

    # Resize and convert to RGB
    img = img.resize(target_size, Image.Resampling.LANCZOS)
    img = img.convert("RGB")

    # Enhance contrast
    contrast_enhancer = ImageEnhance.Contrast(img)
    img = contrast_enhancer.enhance(1.1)

    # Noise reduction
    img = img.filter(ImageFilter.MedianFilter(size=3))

    # Normalize to [0, 1]
    img_array = np.array(img) / 255.0
    return img_array

def load_raw_image(image_path, target_size=(384, 384)):
    """
    Loads the image without preprocessing, only resizing it.
    """
    img = Image.open(image_path)
    img = img.resize(target_size, Image.Resampling.LANCZOS)
    img = img.convert("RGB")
    return np.array(img) / 255.0


In [ ]:
def compare_raw_vs_preprocessed_horizontal(df, num_samples=5):
    plt.figure(figsize=(3 * num_samples, 6))  # Wider, shorter figure

    for i in range(num_samples):
        idx = random.randint(0, len(df) - 1)
        img_path = df.iloc[idx]['image']

        raw_img = load_raw_image(img_path)
        processed_img = preprocess_image(img_path)

        # Top row: Raw image
        plt.subplot(2, num_samples, i + 1)
        plt.imshow(raw_img)
        plt.axis("off")
        plt.title("Raw Image")

        # Bottom row: Preprocessed image
        plt.subplot(2, num_samples, i + 1 + num_samples)
        plt.imshow(processed_img)
        plt.axis("off")
        plt.title("Preprocessed Image")

    plt.tight_layout()
    plt.show()


# Call the function
compare_raw_vs_preprocessed_horizontal(df, num_samples=5)


In [ ]:
def reduce_dataset(df, target_size=20000):
    age_bins = df['age'].unique()
    samples_per_group = target_size // (2 * len(age_bins))  # Half for each gender in each age group
    reduced_df = pd.concat([
        pd.concat([
            df[(df['age'] == age) & (df['gender'] == gender)].sample(min(samples_per_group, len(df[(df['age'] == age) & (df['gender'] == gender)])), random_state=42)
            for gender in [0, 1]
        ]) for age in age_bins
    ])
    return reduced_df.sample(frac=1).reset_index(drop=True)  # Shuffle

# Apply dataset reduction
df = reduce_dataset(df, target_size=20000)  # Change to desired size

gender_dic = {0: 'Male', 1: 'Female'}

In [ ]:
# Visualize the reduced dataset
def visualize_dataset(df):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    df['age'].hist(bins=20, color='skyblue', edgecolor='black')
    plt.xlabel('Age')
    plt.ylabel('Count')
    plt.title('Age Distribution in Reduced Dataset')

    plt.subplot(1, 2, 2)
    df['gender'].value_counts().plot(kind='bar', color=['blue', 'pink'], edgecolor='black')
    plt.xticks(ticks=[0, 1], labels=['Male', 'Female'], rotation=0)
    plt.ylabel('Count')
    plt.title('Gender Distribution in Reduced Dataset')
    plt.show()

visualize_dataset(df)

In [ ]:
# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

def extract_features(df, preprocess=False):
    features, age_labels, gender_labels = [], [], []

    for idx in tqdm(range(len(df))):
        img_path = df.iloc[idx]['image']

        try:
            # Load image with preprocessing if enabled
            img = preprocess_image(img_path) if preprocess else load_raw_image(img_path)

            # Convert image to PIL format & prepare input for CLIP
            inputs = clip_processor(images=Image.fromarray((img * 255).astype(np.uint8)), return_tensors="pt").to(device)

            # Move tensors to device
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Extract features
            with torch.no_grad():
                vec = clip_model.get_image_features(**inputs).squeeze().cpu().numpy()  # Move to CPU before NumPy conversion

            # Append extracted features & labels
            features.append(vec)
            age_labels.append(df.iloc[idx]['age'])
            gender_labels.append(df.iloc[idx]['gender'])

        except Exception as e:
            print(f"⚠️ Error processing {img_path}: {e}")
            continue  # Skip problematic images

    return np.array(features), np.array(age_labels), np.array(gender_labels)


In [ ]:
# Extract features
X_train_raw, y_train_age_raw, y_train_gender_raw = extract_features(train_df, preprocess=False)

import joblib

# Save extracted features
joblib.dump((X_train_raw, y_train_age_raw, y_train_gender_raw), "/content/drive/MyDrive/Age Prediction/Features/X_train_raw.pkl")

print("Features saved successfully!")


In [ ]:
X_test_raw, y_test_age_raw, y_test_gender_raw = extract_features(test_df, preprocess=False)

joblib.dump((X_test_raw, y_test_age_raw, y_test_gender_raw), "/content/drive/MyDrive/Age Prediction/Features/X_test_raw.pkl")

print("Features saved successfully!")

In [ ]:
X_train_pre, y_train_age_pre, y_train_gender_pre = extract_features(train_df, preprocess=True)

# Save preprocessed features
joblib.dump((X_train_pre, y_train_age_pre, y_train_gender_pre), "/content/drive/MyDrive/Age Prediction/Features/X_train_pre.pkl")

print("Features saved successfully!")

In [ ]:
X_test_pre, y_test_age_pre, y_test_gender_pre = extract_features(test_df, preprocess=True)

joblib.dump((X_test_pre, y_test_age_pre, y_test_gender_pre), "/content/drive/MyDrive/Age Prediction/Features/X_test_pre.pkl")

print("Features saved successfully!")

In [ ]:
import joblib

#Load Raw Features
X_train_raw, y_train_age_raw, y_train_gender_raw = joblib.load("/content/drive/MyDrive/Age Prediction/Features/X_train_raw.pkl")
X_test_raw, y_test_age_raw, y_test_gender_raw = joblib.load("/content/drive/MyDrive/Age Prediction/Features/X_test_raw.pkl")

# Load preprocessed features
X_train_pre, y_train_age_pre, y_train_gender_pre = joblib.load("/content/drive/MyDrive/Age Prediction/Features/X_train_pre.pkl")
X_test_pre, y_test_age_pre, y_test_gender_pre = joblib.load("/content/drive/MyDrive/Age Prediction/Features/X_test_pre.pkl")

print("Features loaded successfully!")


In [ ]:
import shap
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name).to(device)
clip_processor = CLIPProcessor.from_pretrained(model_name)

# Function to preprocess images for CLIP
def preprocess_images(images):
    inputs = clip_processor(images=images, return_tensors="pt", padding=True)
    return inputs["pixel_values"].to(device)

# Wrapper function that handles conversion between numpy and torch
def model_predict(images_numpy):
    # Convert numpy array to PyTorch tensor
    images_tensor = torch.tensor(images_numpy, device=device)

    # Get features from CLIP
    with torch.no_grad():
        features = clip_model.get_image_features(images_tensor)

    # Return as numpy for SHAP
    return features.cpu().numpy()

# Load sample images - replace with your own image paths
image_paths = [
    "/root/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/UTKFace/1_0_0_20161219154556757.jpg.chip.jpg",
    "/root/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/UTKFace/45_0_0_20170117135648206.jpg.chip.jpg",
    "/root/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/UTKFace/105_0_0_20170112213001988.jpg.chip.jpg"
]

# Load images
images = [Image.open(path).convert("RGB") for path in image_paths]

# Preprocess the images for CLIP
preprocessed_images = preprocess_images(images)

# Try a different approach with GradientExplainer - this works better with deep models like CLIP
background = preprocessed_images[:1]  # Use first image as background

# Create the wrapper
class CLIPWrapper(torch.nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.clip_model = clip_model

    def forward(self, x):
        # Get a consistent 1D output by averaging the features
        features = self.clip_model.get_image_features(x)
        return features.mean(dim=1, keepdim=True)  # Return shape (batch_size, 1)

# Create gradient explainer wrapper
clip_wrapper = CLIPWrapper(clip_model).to(device)
clip_wrapper.eval()

# Create the explainer
gradient_explainer = shap.GradientExplainer(clip_wrapper, background)

# Generate gradient-based SHAP values
gradient_shap_values = gradient_explainer.shap_values(preprocessed_images)

# Visualize the results
for i, img_path in enumerate(image_paths):
    base_name = img_path.split('/')[-1].split('.')[0]
    img_age = base_name.split('_')[0]  # Extract age from filename

    # Convert SHAP values to numpy for visualization
    # FIX: gradient_shap_values already contains numpy arrays, no need for cpu() or to(device)
    shap_numpy = gradient_shap_values[0][i]

    # Get the shape for reshaping correctly
    input_shape = preprocessed_images[i].cpu().numpy().shape

    # Print more debug info
    print(f"SHAP values shape: {shap_numpy.shape}, input shape: {input_shape}")

    # Based on your output, we know SHAP values are (224, 224, 1)
    if len(shap_numpy.shape) == 3 and shap_numpy.shape[2] == 1:
        # We already have a spatial map, just squeeze the last dimension
        shap_img = shap_numpy[:,:,0]  # Remove singleton dimension
        print(f"Using spatial SHAP map directly")
    elif shap_numpy.size != np.prod(input_shape):
        # If shapes don't match, we need to handle differently
        shap_img = shap_numpy  # Use as is without reshaping
        print(f"Shapes don't match, using raw SHAP values")
    else:
        # Reshape to image dimensions
        shap_img = shap_numpy.reshape(input_shape)
        print(f"Reshaped SHAP values to match input")

    # Take absolute values and normalize for better visualization
    abs_shap = np.abs(shap_img)
    max_val = abs_shap.max()
    if max_val > 0:
        abs_shap = abs_shap / max_val

    # Create visualization
    plt.figure(figsize=(15, 5))

    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(images[i])
    plt.title(f"Original Image (Age: {img_age})")
    plt.axis('off')

    # SHAP values
    plt.subplot(1, 3, 2)

    # Handle different shapes for visualization
    if len(shap_img.shape) == 2:
        # Already 2D - perfect for visualization
        shap_sum = np.abs(shap_img)
    elif len(shap_img.shape) == 3 and shap_img.shape[2] == 1:
        # If we have a 3D tensor with a singleton last dimension
        shap_sum = np.abs(shap_img[:,:,0])
    elif len(shap_img.shape) == 3:
        # If we have 3D tensor (e.g., channels, height, width)
        shap_sum = np.abs(shap_img).sum(axis=0)
    else:
        # If we have a flattened or different shape, reshape for visualization
        # For CLIP feature visualizations, we can just show the raw values
        # or reshape to a square for visualization
        size = int(np.sqrt(abs_shap.size))
        shap_sum = abs_shap.reshape(size, size)

    # Normalize for better visibility
    max_val = shap_sum.max()
    if max_val > 0:
        shap_sum = shap_sum / max_val

    plt.imshow(shap_sum, cmap='hot')
    plt.title(f"SHAP Magnitude (Age: {img_age})")
    plt.axis('off')

    # Overlay
    plt.subplot(1, 3, 3)
    img_array = np.array(images[i])

    # Normalize image to [0, 1]
    img_norm = img_array / 255.0

    # Resize SHAP values to match original image regardless of original shape
    from skimage.transform import resize

    # Make sure shap_sum is 2D for resizing
    if len(shap_sum.shape) != 2:
        print(f"Warning: Expected 2D SHAP values for overlay, got {shap_sum.shape}. Adjusting...")
        # If not 2D, reshape or sum as needed
        if len(shap_sum.shape) > 2:
            shap_sum = shap_sum.sum(axis=tuple(range(len(shap_sum.shape) - 2)))

    # Now resize to match the original image
    shap_resized = resize(shap_sum, (img_array.shape[0], img_array.shape[1]),
                          order=1, mode='constant', anti_aliasing=True)

    # Create a heatmap overlay
    cmap = plt.cm.hot
    shap_heatmap = cmap(shap_resized)[:, :, :3]

    # Combine with alpha blending
    alpha = 0.7
    overlay = alpha * shap_heatmap + (1 - alpha) * img_norm

    plt.imshow(overlay)
    plt.title(f"SHAP Overlay (Age: {img_age})")
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(f"age_{img_age}_clip_explanation.png")
    plt.close()

# Create a comparison visualization across different ages
plt.figure(figsize=(15, 10))

# Get original images in a row
for i, img_path in enumerate(image_paths):
    plt.subplot(2, len(image_paths), i+1)
    plt.imshow(images[i])
    img_age = img_path.split('/')[-1].split('.')[0].split('_')[0]
    plt.title(f"Age: {img_age}")
    plt.axis('off')

# Get SHAP visualizations in a row
for i, img_path in enumerate(image_paths):
    plt.subplot(2, len(image_paths), len(image_paths)+i+1)

    # Convert SHAP values to numpy for visualization
    shap_numpy = gradient_shap_values[0][i]
    print(f"Comparison visualization - SHAP shape: {shap_numpy.shape}")

    # Handle based on the actual shape we're seeing (224, 224, 1)
    if len(shap_numpy.shape) == 3 and shap_numpy.shape[2] == 1:
        # We have a single-channel heatmap already
        abs_shap = np.abs(shap_numpy[:,:,0])  # Remove the singleton dimension
    elif shap_numpy.size != np.prod(preprocessed_images[i].cpu().numpy().shape):
        # If we can't reshape to original dimensions, create a square visualization
        size = int(np.sqrt(shap_numpy.size))
        abs_shap = np.abs(shap_numpy).reshape(size, size)
    else:
        # Reshape to image dimensions
        shap_img = shap_numpy.reshape(preprocessed_images[i].cpu().numpy().shape)
        # Take absolute values and sum across channels
        abs_shap = np.abs(shap_img)
        abs_shap = abs_shap.sum(axis=0)

    # Normalize for better visibility
    max_val = abs_shap.max()
    if max_val > 0:
        norm_shap = abs_shap / max_val
    else:
        norm_shap = abs_shap

    plt.imshow(norm_shap, cmap='hot')
    img_age = img_path.split('/')[-1].split('.')[0].split('_')[0]
    plt.title(f"SHAP (Age: {img_age})")
    plt.axis('off')

plt.suptitle("CLIP Feature Importance Across Different Ages", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.savefig("age_comparison_clip_summary.png")

print("SHAP analysis completed. Images saved with age information.")

# Optional - Feature distribution analysis
print("\nAnalyzing feature distributions across ages...")

# Extract features directly for analysis
with torch.no_grad():
    features = clip_model.get_image_features(preprocessed_images)
    features_np = features.cpu().numpy()

# Create feature distribution visualization
plt.figure(figsize=(12, 6))
for i, img_path in enumerate(image_paths):
    img_age = img_path.split('/')[-1].split('.')[0].split('_')[0]
    plt.plot(features_np[i][:50], label=f"Age {img_age}")  # Plot first 50 features

plt.title("CLIP Feature Distribution Across Ages")
plt.xlabel("Feature Index")
plt.ylabel("Feature Activation")
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig("age_feature_distribution.png")

print("Feature distribution analysis completed.")

In [ ]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

# Load YuNet Model
yunet = cv2.FaceDetectorYN.create(
    model='/content/drive/MyDrive/Age Prediction/Models/face_detection_yunet_2023mar.onnx',
    config='',
    input_size=(320, 320),
    score_threshold=0.9,
    nms_threshold=0.3,
    top_k=5000
)

# Function to apply Grad-CAM++
def apply_gradcam_plusplus(image_path):
    # Read Image
    img = cv2.imread(image_path)
    height, width = img.shape[:2]
    yunet.setInputSize((width, height))

    # Perform Face Detection
    _, faces = yunet.detect(img)

    if faces is None:
        print("❌ No face detected!")
        return

    # Select the first detected face
    x, y, w, h = map(int, faces[0][:4])
    face_crop = img[y:y+h, x:x+w]

    # Convert to Grayscale (Simulating Activation Map for YuNet)
    gray_face = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)

    # Simulating Feature Importance (YuNet doesn’t provide direct feature maps)
    heatmap = cv2.applyColorMap(gray_face, cv2.COLORMAP_JET)
    heatmap = cv2.resize(heatmap, (w, h))

    # Overlay Heatmap
    overlayed = cv2.addWeighted(face_crop, 0.6, heatmap, 0.4, 0)

    # Place back in original image
    img[y:y+h, x:x+w] = overlayed

    # Show Image with Grad-CAM++ Visualization
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Grad-CAM++ for YuNet Face Detection")
    plt.show()

# Run Grad-CAM++ for YuNet
apply_gradcam_plusplus("face_image.jpg")


In [ ]:
from sklearn.manifold import TSNE

def visualize_features(features, labels, title):
    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    reduced_features = tsne.fit_transform(features)

    plt.figure(figsize=(10, 6))
    unique_labels = np.unique(labels)
    for label in unique_labels:
        plt.scatter(reduced_features[labels == label, 0], reduced_features[labels == label, 1], label=label, alpha=0.6)

    plt.legend()
    plt.title(title)
    plt.xlabel("t-SNE Component 1")
    plt.ylabel("t-SNE Component 2")
    plt.show()

In [ ]:
visualize_features(X_train_raw, y_train_age_raw, "t-SNE Visualization of Age Features")
visualize_features(X_train_raw, y_train_gender_raw, "t-SNE Visualization of Gender Features")

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def visualize_features(features, labels, title):
    tsne = TSNE(n_components=3, perplexity=30, random_state=42)
    reduced_features = tsne.fit_transform(features)

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')
    unique_labels = np.unique(labels)
    for label in unique_labels:
        idxs = labels == label
        ax.scatter(reduced_features[idxs, 0], reduced_features[idxs, 1], reduced_features[idxs, 2], label=label, alpha=0.6)

    ax.legend()
    ax.set_title(title)
    ax.set_xlabel("t-SNE Component 1")
    ax.set_ylabel("t-SNE Component 2")
    ax.set_zlabel("t-SNE Component 3")
    plt.show()

In [ ]:
visualize_features(X_train_raw, y_train_age_raw, "3D t-SNE Visualization of Age Features")
visualize_features(X_train_raw, y_train_gender_raw, "3D t-SNE Visualization of Gender Features")

In [ ]:
import plotly.express as px
from sklearn.manifold import TSNE
import numpy as np

def visualize_features(features, labels, title):
    # Perform t-SNE dimensionality reduction to 3 components
    tsne = TSNE(n_components=3, perplexity=30, random_state=42)
    reduced_features = tsne.fit_transform(features)

    # Create a 3D scatter plot using Plotly Express
    fig = px.scatter_3d(
        x=reduced_features[:, 0],
        y=reduced_features[:, 1],
        z=reduced_features[:, 2],
        color=labels.astype(str),  # Convert labels to string for better color mapping
        labels={'x': 't-SNE Component 1', 'y': 't-SNE Component 2', 'z': 't-SNE Component 3'},
        title=title,
        opacity=0.7
    )

    # Update layout for better visualization
    fig.update_layout(
        width=800,
        height=600,
        scene=dict(
            xaxis_title="t-SNE Component 1",
            yaxis_title="t-SNE Component 2",
            zaxis_title="t-SNE Component 3"
        )
    )

    # Display the plot
    fig.show()
    print(f"3D t-SNE for {title}: Rotate to explore clusters!")

# Visualize age features
visualize_features(X_train_raw, y_train_age_raw, "3D t-SNE Visualization of Age Features")

# Visualize gender features
visualize_features(X_train_raw, y_train_gender_raw, "3D t-SNE Visualization of Gender Features")

In [ ]:
def visualize_training_performance(history, title):
    plt.figure(figsize=(10, 5))
    plt.plot(history.loss_curve_, label='Loss Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend()
    plt.show()

def visualize_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(title)
    plt.show()

def train_mlp(X_train, y_train, X_test, y_test, task="age", model_filename="mlp_model.pkl"):
    model = MLPClassifier(
        hidden_layer_sizes=(1024, 512, 256, 128),  # Deeper network
        activation='relu',                   # Can change to 'tanh'
        solver='adam',                        # 'adam' is good for large datasets
        learning_rate_init=0.001,# Adjusts learning rate dynamically
        alpha=0.00001,                        # L2 regularization (not learning rate)
        max_iter=500                          # More training iterations
    )

    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Performance Metrics
    print(f"Accuracy for {task} prediction:", accuracy_score(y_test, y_pred))
    print(f"Classification Report ({task}):\n", classification_report(y_test, y_pred))

    # Visualization Functions (Make sure these exist)
    visualize_confusion_matrix(y_test, y_pred, f"Confusion Matrix - {task} Prediction")
    visualize_training_performance(model, f"Training Performance - {task} Prediction")

    # Save the model
    joblib.dump(model, model_filename)
    print(f"Model saved as {model_filename}")

    return model

In [ ]:
from sklearn.naive_bayes import GaussianNB

def train_naive_bayes(X_train, y_train, X_test, y_test, task="age", model_filename="naive_bayes.pkl"):
    model = GaussianNB()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f"Accuracy for {task} prediction:", accuracy_score(y_test, y_pred))
    print(f"Classification Report ({task}):\n", classification_report(y_test, y_pred))

    visualize_confusion_matrix(y_test, y_pred, f"Confusion Matrix - {task} Prediction")

    joblib.dump(model, model_filename)
    print(f"Model saved as {model_filename}")

    return model


In [ ]:
import joblib
import seaborn as sns

print("\nTraining Age Prediction Model on Raw Images:")
naive_age_raw = train_naive_bayes(X_train_raw, y_train_age_raw, X_test_raw, y_test_age_raw, task="age", model_filename="/content/drive/MyDrive/Age Prediction/Models/naive_age_raw.pkl")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def train_random_forest(X_train, y_train, X_test, y_test, task="age", model_filename="random_forest.pkl"):
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f"Accuracy for {task} prediction:", accuracy_score(y_test, y_pred))
    print(f"Classification Report ({task}):\n", classification_report(y_test, y_pred))

    visualize_confusion_matrix(y_test, y_pred, f"Confusion Matrix - {task} Prediction")

    joblib.dump(model, model_filename)
    print(f"Model saved as {model_filename}")

    return model


In [ ]:
import joblib
import seaborn as sns

print("\nTraining Age Prediction Model on Raw Images:")
rf_age_raw = train_random_forest(X_train_raw, y_train_age_raw, X_test_raw, y_test_age_raw, task="age", model_filename="/content/drive/MyDrive/Age Prediction/Models/rf_age_raw.pkl")

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

def train_adaboost(X_train, y_train, X_test, y_test, task="age", model_filename="adaboost.pkl"):
    model = AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f"Accuracy for {task} prediction:", accuracy_score(y_test, y_pred))
    print(f"Classification Report ({task}):\n", classification_report(y_test, y_pred))

    visualize_confusion_matrix(y_test, y_pred, f"Confusion Matrix - {task} Prediction")

    joblib.dump(model, model_filename)
    print(f"Model saved as {model_filename}")

    return model


In [ ]:
import joblib
import seaborn as sns

print("\nTraining Age Prediction Model on Raw Images:")
mlp_age_raw = train_mlp(X_train_raw, y_train_age_raw, X_test_raw, y_test_age_raw, task="age", model_filename="/content/drive/MyDrive/Age Prediction/Models/mlp_age_raw_adv.pkl")

In [ ]:
print("\nTraining Gender Prediction Model on Raw Images:")
mlp_gender_raw = train_mlp(X_train_raw, y_train_gender_raw, X_test_raw, y_test_gender_raw, task="gender", model_filename="/content/drive/MyDrive/Age Prediction/Models/mlp_gender_raw1_adv.pkl")

In [ ]:
print("\nTraining Age Prediction Model on Preprocessed Images:")
mlp_age_pre = train_mlp(X_train_pre, y_train_age_pre, X_test_pre, y_test_age_pre, task="age", model_filename="/content/drive/MyDrive/Age Prediction/Models/mlp_age_pre1_adv.pkl")

In [ ]:
print("\nTraining Gender Prediction Model on Preprocessed Images:")
mlp_gender_pre = train_mlp(X_train_pre, y_train_gender_pre, X_test_pre, y_test_gender_pre, task="gender", model_filename="/content/drive/MyDrive/Age Prediction/Models/mlp_gender_pre1_adv.pkl")

In [ ]:
import numpy as np
import cv2
import torch
import joblib
import os
from PIL import Image, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
from sklearn.neural_network import MLPClassifier
from collections import defaultdict
from google.colab.patches import cv2_imshow

# Load CLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load pre-trained MLP models for age and gender prediction
age_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_age_raw_adv.pkl")
gender_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_gender_raw1_adv.pkl")

# Initialize YuNet for face detection
yunet = cv2.FaceDetectorYN.create(
    model='/content/drive/MyDrive/Age Prediction/Models/face_detection_yunet_2023mar.onnx',
    config='',
    input_size=(320, 320),
    score_threshold=0.9,
    nms_threshold=0.3,
    top_k=5000
)

def categorize_age(age):
    if age <= 10:
        return "0-10"
    elif age <= 20:
        return "11-20"
    elif age <= 30:
        return "21-30"
    elif age <= 40:
        return "31-40"
    elif age <= 50:
        return "41-50"
    else:
        return "50+"

def preprocess_image(img):
    """Apply DIP techniques: contrast enhancement, noise reduction."""
    img = Image.fromarray(img).convert("RGB")
    img = img.resize((256, 256), Image.Resampling.LANCZOS)
    return np.array(img) / 255.0

def extract_features(img):
    """Extract features using CLIP model."""
    inputs = clip_processor(images=Image.fromarray((img * 255).astype(np.uint8)), return_tensors="pt").to(device)
    with torch.no_grad():
        vec = clip_model.get_image_features(**inputs).squeeze().cpu().numpy()
    return vec

def predict_age_gender(features):
    """Predict age and gender using MLP models."""
    age_label = age_model.predict([features])[0]  # Get predicted age category
    gender_pred = "Male" if gender_model.predict([features])[0] == 0 else "Female"
    return age_label, gender_pred

def process_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        frame = cv2.imread(img_path)
        if frame is None:
            continue

        print(f"\n🔹 Processing Image: {filename}")

        age_groups = defaultdict(int)  # Reset count for each image

        yunet.setInputSize((frame.shape[1], frame.shape[0]))
        _, faces = yunet.detect(frame)  # Detect faces
        frame_raw = frame.copy()
        frame_dip = frame.copy()

        # Raw processing (without DIP)
        frame_raw_resized = cv2.resize(frame_raw, (128, 128)) / 255.0
        features_raw = extract_features(frame_raw_resized)
        age_raw, gender_raw = predict_age_gender(features_raw)
        cv2.putText(frame_raw, f"Raw: {age_raw}, {gender_raw}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Count in age groups
        age_groups[age_raw] += 1

        if faces is not None:
            for face in faces:
                x, y, w, h = map(int, face[:4])
                face_crop = frame[y:y+h, x:x+w]
                face_dip = preprocess_image(face_crop)
                features_dip = extract_features(face_dip)
                age_dip, gender_dip = predict_age_gender(features_dip)
                cv2.rectangle(frame_dip, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame_dip, f"DIP: {age_dip}, {gender_dip}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                # Count in age groups (DIP method)
                age_groups[age_dip] += 1

        combined_frame = np.hstack((frame_raw, frame_dip))
        cv2_imshow(combined_frame)

        # Print Summary **for the current image**
        print("\n📊 **Summary for Image:**")
        for group, count in age_groups.items():
            print(f"{group} years: {count} people")

if __name__ == "__main__":
    folder_path = "/content/drive/MyDrive/Age Prediction/Real Time Test Data"  # Folder path
    process_images_in_folder(folder_path)  # Run the function


In [ ]:
import numpy as np
import cv2
import torch
import joblib
import os
from PIL import Image, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
from sklearn.neural_network import MLPClassifier
from collections import defaultdict
from google.colab.patches import cv2_imshow

# Load CLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load pre-trained MLP models for age and gender prediction
age_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_age_raw_adv.pkl")
gender_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_gender_raw1_adv.pkl")

# Initialize YuNet for face detection
yunet = cv2.FaceDetectorYN.create(
    model='/content/drive/MyDrive/Age Prediction/Models/face_detection_yunet_2023mar.onnx',
    config='',
    input_size=(320, 320),
    score_threshold=0.9,
    nms_threshold=0.3,
    top_k=5000
)

def categorize_age(age):
    if age <= 10:
        return "0-10"
    elif age <= 20:
        return "11-20"
    elif age <= 30:
        return "21-30"
    elif age <= 40:
        return "31-40"
    elif age <= 50:
        return "41-50"
    else:
        return "50+"

def preprocess_image(img):
    """Apply DIP techniques: contrast enhancement, noise reduction."""
    img = Image.fromarray(img).convert("RGB")
    img = img.resize((256, 256), Image.Resampling.LANCZOS)
    return np.array(img) / 255.0

def extract_features(img):
    """Extract features using CLIP model."""
    inputs = clip_processor(images=Image.fromarray((img * 255).astype(np.uint8)), return_tensors="pt").to(device)
    with torch.no_grad():
        vec = clip_model.get_image_features(**inputs).squeeze().cpu().numpy()
    return vec

def predict_age_gender(features):
    """Predict age and gender using MLP models."""
    age_label = age_model.predict([features])[0]  # Get predicted age category
    gender_pred = "Male" if gender_model.predict([features])[0] == 0 else "Female"
    return age_label, gender_pred

def process_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        frame = cv2.imread(img_path)
        if frame is None:
            continue

        print(f"\n🔹 Processing Image: {filename}")

        age_groups = defaultdict(int)  # Reset count for each image

        yunet.setInputSize((frame.shape[1], frame.shape[0]))
        _, faces = yunet.detect(frame)  # Detect faces
        frame_raw = frame.copy()
        frame_dip = frame.copy()

        # Raw processing (without DIP)
        frame_raw_resized = cv2.resize(frame_raw, (128, 128)) / 255.0
        features_raw = extract_features(frame_raw_resized)
        age_raw, gender_raw = predict_age_gender(features_raw)
        cv2.putText(frame_raw, f"Raw: {age_raw}, {gender_raw}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Count in age groups
        age_groups[age_raw] += 1

        if faces is not None:
            for face in faces:
                x, y, w, h = map(int, face[:4])
                face_crop = frame[y:y+h, x:x+w]
                face_dip = preprocess_image(face_crop)
                features_dip = extract_features(face_dip)
                age_dip, gender_dip = predict_age_gender(features_dip)
                cv2.rectangle(frame_dip, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame_dip, f"DIP: {age_dip}, {gender_dip}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                # Count in age groups (DIP method)
                age_groups[age_dip] += 1

        combined_frame = np.hstack((frame_raw, frame_dip))
        cv2_imshow(combined_frame)

        # Print Summary **for the current image**
        print("\n📊 **Summary for Image:**")
        for group, count in age_groups.items():
            print(f"{group} years: {count} people")

if __name__ == "__main__":
    folder_path = "/content/drive/MyDrive/Age Prediction/Real Time Test Data/Test 1"  # Folder path
    process_images_in_folder(folder_path)  # Run the function


In [ ]:
import numpy as np
import cv2
import torch
import joblib
import os
from PIL import Image, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
from sklearn.neural_network import MLPClassifier
from collections import defaultdict
from google.colab.patches import cv2_imshow

# Load CLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load pre-trained MLP models for age and gender prediction
age_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_age_raw_adv.pkl")
gender_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_gender_raw1_adv.pkl")

# Initialize YuNet for face detection
yunet = cv2.FaceDetectorYN.create(
    model='/content/drive/MyDrive/Age Prediction/Models/face_detection_yunet_2023mar.onnx',
    config='',
    input_size=(320, 320),
    score_threshold=0.9,
    nms_threshold=0.3,
    top_k=5000
)

def categorize_age(age):
    if age <= 10:
        return "0-10"
    elif age <= 20:
        return "11-20"
    elif age <= 30:
        return "21-30"
    elif age <= 40:
        return "31-40"
    elif age <= 50:
        return "41-50"
    else:
        return "50+"

def preprocess_image(img):
    """Apply DIP techniques: contrast enhancement, noise reduction."""
    img = Image.fromarray(img).convert("RGB")
    img = img.resize((256, 256), Image.Resampling.LANCZOS)
    return np.array(img) / 255.0

def extract_features(img):
    """Extract features using CLIP model."""
    inputs = clip_processor(images=Image.fromarray((img * 255).astype(np.uint8)), return_tensors="pt").to(device)
    with torch.no_grad():
        vec = clip_model.get_image_features(**inputs).squeeze().cpu().numpy()
    return vec

def predict_age_gender(features):
    """Predict age and gender using MLP models."""
    age_label = age_model.predict([features])[0]  # Get predicted age category
    gender_pred = "Male" if gender_model.predict([features])[0] == 0 else "Female"
    return age_label, gender_pred

def process_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        frame = cv2.imread(img_path)
        if frame is None:
            continue

        print(f"\n🔹 Processing Image: {filename}")

        age_groups = defaultdict(int)  # Reset count for each image

        yunet.setInputSize((frame.shape[1], frame.shape[0]))
        _, faces = yunet.detect(frame)  # Detect faces
        frame_raw = frame.copy()
        frame_dip = frame.copy()

        # Raw processing (without DIP)
        frame_raw_resized = cv2.resize(frame_raw, (128, 128)) / 255.0
        features_raw = extract_features(frame_raw_resized)
        age_raw, gender_raw = predict_age_gender(features_raw)
        cv2.putText(frame_raw, f"Raw: {age_raw}, {gender_raw}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Count in age groups
        age_groups[age_raw] += 1

        if faces is not None:
            for face in faces:
                x, y, w, h = map(int, face[:4])
                face_crop = frame[y:y+h, x:x+w]
                face_dip = preprocess_image(face_crop)
                features_dip = extract_features(face_dip)
                age_dip, gender_dip = predict_age_gender(features_dip)
                cv2.rectangle(frame_dip, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame_dip, f"DIP: {age_dip}, {gender_dip}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                # Count in age groups (DIP method)
                age_groups[age_dip] += 1

        combined_frame = np.hstack((frame_raw, frame_dip))
        cv2_imshow(combined_frame)

        # Print Summary **for the current image**
        print("\n📊 **Summary for Image:**")
        for group, count in age_groups.items():
            print(f"{group} years: {count} people")

if __name__ == "__main__":
    folder_path = "/content/drive/MyDrive/Age Prediction/Real Time Test Data/Test 2"  # Folder path
    process_images_in_folder(folder_path)  # Run the function


In [ ]:
import numpy as np
import cv2
import torch
import joblib
import os
from PIL import Image, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
from sklearn.neural_network import MLPClassifier
from collections import defaultdict
from google.colab.patches import cv2_imshow

# Load CLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load pre-trained MLP models for age and gender prediction
age_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_age_raw_adv.pkl")
gender_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_gender_raw1_adv.pkl")

# Initialize YuNet for face detection
yunet = cv2.FaceDetectorYN.create(
    model='/content/drive/MyDrive/Age Prediction/Models/face_detection_yunet_2023mar.onnx',
    config='',
    input_size=(320, 320),
    score_threshold=0.9,
    nms_threshold=0.3,
    top_k=5000
)

def categorize_age(age):
    if age <= 10:
        return "0-10"
    elif age <= 20:
        return "11-20"
    elif age <= 30:
        return "21-30"
    elif age <= 40:
        return "31-40"
    elif age <= 50:
        return "41-50"
    else:
        return "50+"

def preprocess_image(img):
    """Apply DIP techniques: contrast enhancement, noise reduction."""
    img = Image.fromarray(img).convert("RGB")
    img = img.resize((256, 256), Image.Resampling.LANCZOS)
    return np.array(img) / 255.0

def extract_features(img):
    """Extract features using CLIP model."""
    inputs = clip_processor(images=Image.fromarray((img * 255).astype(np.uint8)), return_tensors="pt").to(device)
    with torch.no_grad():
        vec = clip_model.get_image_features(**inputs).squeeze().cpu().numpy()
    return vec

def predict_age_gender(features):
    """Predict age and gender using MLP models."""
    age_label = age_model.predict([features])[0]  # Get predicted age category
    gender_pred = "Male" if gender_model.predict([features])[0] == 0 else "Female"
    return age_label, gender_pred

def process_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        frame = cv2.imread(img_path)
        if frame is None:
            continue

        print(f"\n🔹 Processing Image: {filename}")

        age_gender_count = defaultdict(lambda: defaultdict(int))  # Nested dictionary for age and gender counts

        yunet.setInputSize((frame.shape[1], frame.shape[0]))
        _, faces = yunet.detect(frame)  # Detect faces
        frame_raw = frame.copy()
        frame_dip = frame.copy()

        # Raw processing (without DIP)
        frame_raw_resized = cv2.resize(frame_raw, (128, 128)) / 255.0
        features_raw = extract_features(frame_raw_resized)
        age_raw, gender_raw = predict_age_gender(features_raw)
        cv2.putText(frame_raw, f"Raw: {age_raw}, {gender_raw}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Count in age-gender groups
        age_gender_count[age_raw][gender_raw] += 1

        if faces is not None:
            for face in faces:
                x, y, w, h = map(int, face[:4])
                face_crop = frame[y:y+h, x:x+w]
                face_dip = preprocess_image(face_crop)
                features_dip = extract_features(face_dip)
                age_dip, gender_dip = predict_age_gender(features_dip)
                cv2.rectangle(frame_dip, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame_dip, f"DIP: {age_dip}, {gender_dip}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                # Count in age-gender groups (DIP method)
                age_gender_count[age_dip][gender_dip] += 1

        combined_frame = np.hstack((frame_raw, frame_dip))
        cv2_imshow(combined_frame)

        # Print Summary **for the current image**
        print("\n📊 **Summary for Image:**")
        for age_group, gender_dict in age_gender_count.items():
            for gender, count in gender_dict.items():
                print(f"{age_group} years ({gender}): {count} people")

    cv2.destroyAllWindows()

if __name__ == "__main__":
    folder_path = "/content/drive/MyDrive/Age Prediction/Real Time Test Data"  # Folder path
    process_images_in_folder(folder_path)


In [ ]:
import numpy as np
import cv2
import torch
import joblib
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from google.colab.patches import cv2_imshow
from IPython.display import HTML
from base64 import b64encode

# Load CLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load pre-trained MLP models
age_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_age_raw_adv.pkl")
gender_model = joblib.load("/content/drive/MyDrive/Age Prediction/Models/mlp_gender_raw1_adv.pkl")

# Initialize YuNet for face detection
yunet = cv2.FaceDetectorYN.create(
    model='/content/drive/MyDrive/Age Prediction/Models/face_detection_yunet_2023mar.onnx',
    config='',
    input_size=(320, 320),
    score_threshold=0.9,
    nms_threshold=0.3,
    top_k=5000
)

# Age and gender count storage
age_distribution = {}
gender_count = {"Male": 0, "Female": 0}

def preprocess_image(img):
    """Preprocess the image: Resize & Normalize"""
    img = Image.fromarray(img).convert("RGB")
    img = img.resize((384, 384), Image.Resampling.LANCZOS)
    return np.array(img) / 255.0  # Normalize to [0,1]

def extract_features(img):
    """Extract features using CLIP"""
    inputs = clip_processor(images=Image.fromarray((img * 255).astype(np.uint8)), return_tensors="pt").to(device)
    with torch.no_grad():
        vec = clip_model.get_image_features(**inputs).squeeze().cpu().numpy()
    return vec

def predict_age_gender(features):
    """Predict age and gender using MLP models"""
    age_label = age_model.predict([features])[0]  # Age category
    gender_pred = "Male" if gender_model.predict([features])[0] == 0 else "Female"
    return age_label, gender_pred

def process_video(video_path, output_path="output_video.mp4"):
    global age_distribution, gender_count
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ Error: Could not open video file")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        yunet.setInputSize((frame.shape[1], frame.shape[0]))
        _, faces = yunet.detect(frame)

        if faces is not None:
            for face in faces:
                x, y, w, h = map(int, face[:4])
                face_crop = frame[y:y+h, x:x+w]
                face_dip = preprocess_image(face_crop)
                features_dip = extract_features(face_dip)
                age_dip, gender_dip = predict_age_gender(features_dip)

                # Update count
                gender_count[gender_dip] += 1
                age_distribution[age_dip] = age_distribution.get(age_dip, 0) + 1

                # Draw bounding box & overlay predictions
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, f"{age_dip}, {gender_dip}", (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # Display summary at top-left corner
        summary_text = f"Male: {gender_count['Male']} | Female: {gender_count['Female']}"
        cv2.putText(frame, summary_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        age_text = "Age Dist: " + ", ".join([f"{k}: {v}" for k, v in age_distribution.items()])
        cv2.putText(frame, age_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        out.write(frame)
        frame_count += 1
        if frame_count % 10 == 0:
            print(f"🔹 Processed {frame_count} frames...")

    cap.release()
    out.release()
    print("✅ Video processing complete!")

def display_video(video_path):
    """Display video in Google Colab"""
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'<video width=700 controls><source src="{data_url}" type="video/mp4"></video>')

if __name__ == "__main__":
    video_path = "/content/drive/MyDrive/Age Prediction/Real Time Test Data/test_video1 - Made with Clipchamp_1741793443494.mp4"
    output_path = "/content/drive/MyDrive/Age Prediction/processed_video.mp4"

    process_video(video_path, output_path)
    display_video(output_path)


In [ ]:
def display_video(video_path):
    """Display video in Google Colab"""
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""<video width=700 controls><source src="{data_url}" type="video/mp4"></video>""")

if __name__ == "__main__":
    video_path = "/content/drive/MyDrive/Age Prediction/processed_video.mp4"
    display_video(video_path)